# **1. Overview**

Profiling and benchmark models can serve different purposes. We here provide an overview of some methodologies supported in SpeechBrain.

**We encourage our reader to first take a detailed look at all the basics tutorials on SpeechBrain before jumping here**



### **1.1 Introduction to profiling utils**
PyTorch profiling supports many functionalities. Here we will introduce some of them:

1. Decoration of SpeechBrain classes**
```python
@export    # optional: write benchmarks to a log folder
@schedule  # optional: let caching mechanisms warm-up, so that effective demands are recorded only
@profile   # basic profiler - starts recording right away (no warm-up) and must be used right away (no disk I/O)
class SimpleBrain(Brain):
    def compute_forward(self, batch, stage):
        return self.modules.model(batch[0])

    def compute_objectives(self, predictions, batch, stage):
        return torch.nn.functional.l1_loss(predictions, batch[1])
```
  `@profile` has the following properties:
  - The PyTorch profiler instance is accessible: `SimpleBrain.profiler`
  - Profiling starts from scratch at each interaction with a `SimpleBrain` instance
    - If an `@scheduler` is set, the scheduler is reseted as well: starting from zero, thus needing to warm-up again.
    - Previously recorded `FunctionEvents` are stored in raw form in `SimpleBrain.profiler.speechbrain_event_traces`
  - Methods of `SimpleBrain` are hooked to the profiler; by-default these are: `fit()` and `evaluate()`.
2. Decorate a function
```python
@profile
def train(brain, train_set, valid_set):
       brain.fit(epoch_counter=range(10), train_set=train_set, valid_set=valid_set)
```
  For a `void` function w/o `return`, the PyTorch profiler is returned because of the `@profiler` decorator:
  ```python
untracked_brain = SimpleBrain({"model": model}, lambda x: SGD(x, 0.1), run_opts={"device": device})
prof = train(untracked_brain, training_set, validation_set)
  ```
  This way, the instance `untracked_brain` has no attribute `profiler`.
3. Profile a pretrained model
```python
asr_model = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-crdnn-rnnlm-librispeech",
    savedir="pretrained_models/asr-crdnn-rnnlm-librispeech",
)
profile(asr_model, class_hooks=["transcribe_batch"])
```
  This way, the instance `asr_model` has the attribute `asr_model.profiler` (as above) and only the function `transcribe_batch()` is hooked to the PyTorch profiler. <br/><i>Note: `transcribe_batch()` is called by all other interface functions of `EncoderDecoderASR`.</i>
4. SpeechBrain provides default profilers with pre-configured `@schedule` and `@export` functionality:
  - `@profile_optimiser` which also records modules and memory<br/>(faster)
  - `@profile_analyst` which further records shapes, source information, and estimates FLOPs—at the expense of computational overheads<br/>(slower)
  - `@profile_report` which is set-up like `@profile_optimiser` but with a scheduler that allows to estimate upper control limits to real-time factors.

Next, we cover full examples including benchmark printing and visualisation.

First, let's set-up our environment :)

In [ ]:

# Local installation
!git clone https://github.com/TParcollet/speechbrain-released/
%cd /content/speechbrain-released/
!pip install -r requirements.txt
!pip install -e .

# As an alterative, use the PyPI installation
# !pip install speechbrain

Cloning into 'speechbrain-released'...
remote: Enumerating objects: 60719, done.
remote: Counting objects: 100% (1188/1188), done.
remote: Compressing objects: 100% (537/537), done.
remote: Total 60719 (delta 722), reused 1048 (delta 627), pack-reused 59531
Receiving objects: 100% (60719/60719), 62.76 MiB | 24.26 MiB/s, done.
Resolving deltas: 100% (39431/39431), done.
/content/speechbrain-released
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring SoundFile: markers 'sys_platform == "win32"' don't match your environment
     |████████████████████████████████| 97 kB 4.6 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 69 kB 8.7 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
     |████████████████████████████████| 246 kB 41.9 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
     |████████████████████████████████| 101 kB 11.1 MB/s 
     |██████████████

# **2. Benchmarking a pretrained model**



This example uses an `EncoderDecoderASR` and covers three use cases using audio data for `@profile`, `@profile_optimiser`, and `@profile_analyst`:
1. Profile CPU time for single audio    w/o scheduler; no report exporting
2. Real-time measurement (profiling includes PyTorch optimisation)<br/><i>w/ scheduler; w/ tensorboard export</i>
3. Analyse details (adds overheads)<br/><i>w/ scheduler; w/ tensorboard export — adds: FlameGraph data export & FLOPs estimation</i>


In [ ]:
from speechbrain.pretrained import EncoderDecoderASR
from speechbrain.utils.profiling import (
    profile,
    profile_analyst,
    profile_optimiser,
    report_time,
)

asr_model = EncoderDecoderASR.from_hparams(
    source="speechbrain/asr-crdnn-rnnlm-librispeech",
    savedir="pretrained_models/asr-crdnn-rnnlm-librispeech",
)

funcs_to_profile = ["transcribe_batch"]  # because of EncoderDecoderASR type

Downloading:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/212M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253k [00:00<?, ?B/s]

## **2.1 Profile time for a single audio**

Use of basic profiler. For one audio file, the time measurement is reported. This time measurement depends on the settings of the profiler. Here, all (function) events of `transcribe_batch()` (invoked by `transcribe_file()`, are recorded, including overheads resulting from first time use. 

In [ ]:
from copy import deepcopy


# by-value initialisation for clean demonstration of different use cases
asr_copy = deepcopy(asr_model)

# declare specific function for profiling
profile(asr_copy, class_hooks=funcs_to_profile)

# runs benchmark also
asr_copy.transcribe_file("speechbrain/asr-crdnn-rnnlm-librispeech/example.wav")

# prints formatted CPU & CUDA time and returns: cpu_time, cuda_time (total time)
report_time(asr_copy, verbose=False)

# prints details by PyTorch function
print(asr_copy.profiler.key_averages().table(
    sort_by="cpu_time_total", 
    row_limit=10
    ))


Downloading:   0%|          | 0.00/104k [00:00<?, ?B/s]

Comparing the above figures to an offline run on laptop CPU, `report_time(asr_copy)` resulted in:
> Self CPU time total: 19.260s

The PyTorch profiler prints benchmark for:
* `Self CPU` and `Self CPU %`<br/><i>Let's take the `aten::linear` function as an example—on laptop CPU, it computed for 43.107ms (0.22% of 19.260s) taking together all of its calls during `transcribe_batch()`, which excludes waiting times for [e.g.] nested functions to complete.</i>
* `CPU total` and `CPU total %`<br/><i>Once more the `aten::linear` function on laptop CPU, all of its computational contexts were active for 7.825s (40.63% of 19.260s) until [e.g.] nested functions had completed.</i>
* `# of Calls` and `CPU time avg`<br/><i>For `aten::linear` on laptop CPU, it was called 981 times (averaging 7.825s to 7.976ms).</i>

For function events, the `Self CPU` is about computation time and `CPU total` is about context lifespan. A context's lifespan is at least as long as its 'self" computation time.

## **2.2 Real-time measurement for a set of audios**

To get a representative real-time estimate, the profiler needs to be scheduled.



### 2.2.1 Intermezzo: scheduling a PyTorch profiler
The scheduler is a function that returns a `ProfilerAction` (enum type) for the step number and phase the profiler is in. These returns can look like this:
```python
  00 = {ProfilerAction} ProfilerAction.NONE
  01 = {ProfilerAction} ProfilerAction.NONE
  02 = {ProfilerAction} ProfilerAction.WARMUP
  03 = {ProfilerAction} ProfilerAction.WARMUP
  04 = {ProfilerAction} ProfilerAction.RECORD
  05 = {ProfilerAction} ProfilerAction.RECORD_AND_SAVE
  06 = {ProfilerAction} ProfilerAction.NONE
  07 = {ProfilerAction} ProfilerAction.NONE
  08 = {ProfilerAction} ProfilerAction.NONE
  09 = {ProfilerAction} ProfilerAction.NONE
```

As for this scheduler setting, at least five batches are required before anything happens. How to configure a scheduler is explained later; for now, let's just take this as a pre-configured setting :)

In SpeechBrain, the profiler's steps are incremented after processing each batch in `fit()` (for training and validation data( and in `evaluate()` (for test data).



###2.2.2 Intermezzo: pretrained models and profiler.step()

Let's take a look first at two code snippets:
* how are batches processed in the pretrained model
* how is the targeted function hooked by our profiling utils

The goal is to know where to increment the `profiler.step_num` via `profiler.step()` which is in turn input to the scheduler.


> see: EncoderDecoderASR.transcribe_batch()
```python
with torch.no_grad():                                                       # not relevant to: profiler.step()
    wav_lens = wav_lens.to(self.device)                                     # no indicator of a 'step'
    encoder_out = self.encode_batch(wavs, wav_lens)                         # processes a batch at-once
    predicted_tokens, scores = self.mods.decoder(encoder_out, wav_lens)     # ... continues with that batch
    predicted_words = [                                                     # ... continues with that batch
        self.tokenizer.decode_ids(token_seq)                                # 〃
        for token_seq in predicted_tokens                                   # 〃
    ]                                                                       # 〃
return predicted_words, predicted_tokens                                    # end.
```

Consequently, there is no clear way to increment profiler.step_num in-between.

When a function `f()` is hooked by `@profile`, the following happens:

> see: speechbrain.utils.profiling.hook_brain.hook()
```python
prof.start()                    # profiling begins - profiler.step_num = 0
r = f(*f_args, **f_kwargs)      # profiling runs - scheduled warm-up & recording - profiler.step() to be called
prof.stop()                     # profiling ends - results are gathered and written
return r                        # end.
```

Moreover, in absense of a function that handles batch iterations and `profiler.step()` calls, hooking a pretrained model is without impact. Consequently, a better way to go is to treat pretrained models as the `untracked_brain` above.

Let's try a different way to use the polymorphic decorators :)

### 2.2.3 Scheduling profilers for any pretrained model with @profile_optimiser

First, we create a list of audios (later, we demonstrate how to use mock-up data instead).

In [ ]:
# Example: prepare a batch sampler - here: naive 1-by-1 sampling (to get a handle on the profiler's step counter).
example_path = "tests/samples/ASR/"
example_wavs = [
    example_path + "spk1_snt1.wav",
    example_path + "spk1_snt2.wav",
    example_path + "spk1_snt3.wav",
    example_path + "spk1_snt4.wav",
    example_path + "spk1_snt5.wav",
    example_path + "spk1_snt6.wav",
]

Then, we define the test loop: `@profile_optimiser` as a scheduler and we 
create the pre-configured profiler directly as `profile_optimiser()`.

In [ ]:
import torch
from speechbrain.dataio.dataio import read_audio

# Profiling activity.
duration = []
with profile_optimiser() as prof:  # not covered by unittests - since neither a
    for wav in example_wavs:  # as mentioned, naive sampling (1-by1) - use instead: padding, collate_fn & batching
        signal = read_audio(wav)
        scheduler_action = prof.schedule(prof.step_num)
        if (scheduler_action == torch.profiler.ProfilerAction.RECORD) or (scheduler_action == torch.profiler.ProfilerAction.RECORD_AND_SAVE):
          duration.append(signal.shape[0] / 16000)  # fs=16 kHz
        asr_model.transcribe_batch(signal.unsqueeze(0), torch.tensor([1.0]))
        prof.step()

print(prof.profiler.key_averages().table(sort_by="cpu_time_total", row_limit=10))

--------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
--------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         ProfilerStep*         1.64%     258.883ms       100.00%       15.746s        7.873s      18.16 Mb      -3.49 Gb             2  
                          aten::linear         0.12%      18.548ms        81.44%       12.823s       8.381ms       1.26 Gb           0 b          1530  
                            aten::lstm         0.65%     102.045ms        69.66%       10.969s     274.228ms     119.83 Mb    -644.10 Mb            40  
                           aten::addmm        52.75%        8.306s        53.29%  

The `ProfilerStep*` function matches the consumption of the recorded functions; this line can be ignored. `@profile_optimiser` records memory usage as `Self CPU Mem` (own allocations; negative if external to profiling) and `CPU Mem` (during the context lifespan of a function).

Instead of the comprehensive printout, a brief reporting is available.

In [ ]:
bytes_in_GB = 1024.0**3
print('The scheduler recorded total memory load: %.2f GB' % (prof.profiler.total_average().cpu_memory_usage / bytes_in_GB))
# here, two audios were profiled in two batches - at different memory needs

cpu_time, cuda_time = report_time(prof, verbose=False)  # in us
print('The scheduler recorded for audios with total duration {}s - per audio: {}'.format(sum(duration), duration))

us_in_s = 1000.0 * 1000.0
real_time_factor = cpu_time / us_in_s / sum(duration)
print('Real-time factor: %.4f' % real_time_factor)


The scheduler recorded total memory load: 7.92 GB
The scheduler recorded for audios with total duration 4.890000000000001s - per audio: [2.6, 2.29]
Real-time factor: 3.2200


For comparison, on laptop CPU, the real-time factor is ~1.6122 with:
> Self CPU time total: 14.687s

with `aten::linear` at 26.458ms (self CPU); 5.400ms (time avg); 1.07 Gb (Mem), and 2132 calls. For the single audio case w/o scheduler, the avg CPU time was 6.285ms—the scheduler avoids recording overheads from warm-up.

## 2.3 More details on overheads with @profile_analyst

To record more, at the expense of overheads, some PyTorch optimisations are dropped for detailed recording. The generated table covers only (matrix multiplication & conv2d) MFLOPs as additional information.

In [ ]:
with profile_analyst() as analyst:
    for wav in example_wavs:
        signal = read_audio(wav)
        asr_model.transcribe_batch(signal.unsqueeze(0), torch.tensor([1.0]))
        analyst.step()  # NB: will also report Total MFLOPs

print(analyst.profiler.key_averages().table(sort_by="cpu_time_total"))

--------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total MFLOPs  
--------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         ProfilerStep*         1.86%     290.701ms       100.00%       15.614s        7.807s      19.45 Mb      -3.48 Gb             2            --  
                          aten::linear         0.19%      29.215ms        80.78%       12.613s       8.244ms       1.26 Gb           0 b          1530            --  
                            aten::lstm         0.78%     121.878ms        69.94%       10.920s     273.000ms     119.83 Mb    -644.38 Mb            40            -- 

To print recorded input shapes:

In [ ]:
print(analyst.profiler.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=25))

--------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls                                                                      Input Shapes   Total FLOPs  
--------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  ------------  
                         ProfilerStep*         1.86%     290.701ms       100.00%       15.614s        7.807s      19.45 Mb      -3.48 Gb             2                                                                                []            --  
    

Summary: by grouping averages without considering input shapes, we get a gross estimate. By grouping averages with input shapes in mind, we get more insights into the inner workings of a model - this comes though at the expense of computation time. On laptop CPU:
> @profile_optimiser => Self CPU time total: 14.687s
> 
> @profile_analyst   => Self CPU time total: 19.427s

# 3. Profiling `fit()` and `evaluate()` at minimal invasiveness

Long story short, we modify the code of this integration test:
[tests/integration/neural_networks/ASR_seq2seq/example_asr_seq2seq_experiment.py](https://github.com/speechbrain/speechbrain/blob/main/tests/integration/neural_networks/ASR_seq2seq/example_asr_seq2seq_experiment.py)

To the data pipeline and class implementations, one single line of code is added (if benchmark results should be printed only):
```python
@profile_optimiser  # <== this one
class seq2seqBrain(sb.Brain):
  ...
```
All of the rest is copy pasted.

The full code example:

In [ ]:
import pathlib
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml
from speechbrain.utils.profiling import profile_optimiser, export  # import added ;-)


@export  # optional, here for demonstrating to save traces into a log folder
@profile_optimiser
class seq2seqBrain(sb.Brain):
    def compute_forward(self, batch, stage):
        "Given an input batch it computes the output probabilities."
        batch = batch.to(self.device)
        wavs, wav_lens = batch.sig
        phns_bos, _ = batch.phn_encoded_bos
        feats = self.hparams.compute_features(wavs)
        feats = self.modules.mean_var_norm(feats, wav_lens)
        x = self.modules.enc(feats)

        # Prepend bos token at the beginning
        e_in = self.modules.emb(phns_bos)
        h, w = self.modules.dec(e_in, x, wav_lens)
        logits = self.modules.lin(h)
        outputs = self.hparams.softmax(logits)

        if stage != sb.Stage.TRAIN:
            seq, _ = self.hparams.searcher(x, wav_lens)
            return outputs, seq

        return outputs

    def compute_objectives(self, predictions, batch, stage):
        "Given the network predictions and targets computed the NLL loss."
        if stage == sb.Stage.TRAIN:
            outputs = predictions
        else:
            outputs, seq = predictions

        ids = batch.id
        phns, phn_lens = batch.phn_encoded_eos

        loss = self.hparams.compute_cost(outputs, phns, length=phn_lens)

        if stage != sb.Stage.TRAIN:
            self.per_metrics.append(ids, seq, phns, target_len=phn_lens)

        return loss

    def fit_batch(self, batch):
        """Fits train batches"""
        preds = self.compute_forward(batch, sb.Stage.TRAIN)
        loss = self.compute_objectives(preds, batch, sb.Stage.TRAIN)
        loss.backward()
        if self.check_gradients(loss):
            self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.detach()

    def evaluate_batch(self, batch, stage=sb.Stage.TEST):
        """Evaluates test batches"""
        out = self.compute_forward(batch, stage)
        loss = self.compute_objectives(out, batch, stage)
        return loss.detach()

    def on_stage_start(self, stage, epoch=None):
        "Gets called when a stage (either training, validation, test) starts."
        if stage != sb.Stage.TRAIN:
            self.per_metrics = self.hparams.per_stats()

    def on_stage_end(self, stage, stage_loss, epoch=None):
        "Gets called when a stage (either training, validation, test) ends."
        if stage == sb.Stage.TRAIN:
            self.train_loss = stage_loss
        if stage == sb.Stage.VALID and epoch is not None:
            print("Epoch %d complete" % epoch)
            print("Train loss: %.2f" % self.train_loss)
        if stage != sb.Stage.TRAIN:
            print(stage, "loss: %.2f" % stage_loss)
            print(stage, "PER: %.2f" % self.per_metrics.summarize("error_rate"))


def data_prep(data_folder, hparams):
    "Creates the datasets and their data processing pipelines."

    # 1. Declarations:
    train_data = sb.dataio.dataset.DynamicItemDataset.from_json(
        json_path="tests/samples/annotation/ASR_train.json",
        replacements={"data_root": data_folder},
    )
    valid_data = sb.dataio.dataset.DynamicItemDataset.from_json(
        json_path="tests/samples/annotation/ASR_dev.json",
        replacements={"data_root": data_folder},
    )
    datasets = [train_data, valid_data]
    label_encoder = sb.dataio.encoder.TextEncoder()

    # 2. Define audio pipeline:
    @sb.utils.data_pipeline.takes("wav")
    @sb.utils.data_pipeline.provides("sig")
    def audio_pipeline(wav):
        sig = sb.dataio.dataio.read_audio(wav)
        return sig

    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline)

    # 3. Define text pipeline:
    @sb.utils.data_pipeline.takes("phn")
    @sb.utils.data_pipeline.provides(
        "phn_list", "phn_encoded_bos", "phn_encoded_eos"
    )
    def text_pipeline(phn):
        phn_list = phn.strip().split()
        yield phn_list
        phn_encoded = label_encoder.encode_sequence_torch(phn_list)
        phn_encoded_bos = label_encoder.prepend_bos_index(phn_encoded).long()
        yield phn_encoded_bos
        phn_encoded_eos = label_encoder.append_eos_index(phn_encoded).long()
        yield phn_encoded_eos

    sb.dataio.dataset.add_dynamic_item(datasets, text_pipeline)

    # 3. Fit encoder:
    # NOTE: In this minimal example, also update from valid data
    label_encoder.insert_bos_eos(bos_index=hparams["bos_index"])
    label_encoder.update_from_didataset(train_data, output_key="phn_list")
    label_encoder.update_from_didataset(valid_data, output_key="phn_list")

    # 4. Set output:
    sb.dataio.dataset.set_output_keys(
        datasets, ["id", "sig", "phn_encoded_eos", "phn_encoded_bos"]
    )
    return train_data, valid_data


Data preparation:

In [ ]:
device = "cpu"  # from test main() input args

# Paths adjusted
experiment_dir = pathlib.Path(".").resolve()
hparams_file = "tests/integration/ASR_seq2seq/hyperparams.yaml"
data_folder = "tests/samples/ASR"
data_folder = (experiment_dir / data_folder).resolve()

# Load model hyper parameters:
with open(hparams_file) as fin:
    hparams = load_hyperpyyaml(fin)

# Dataset creation
train_data, valid_data = data_prep(data_folder, hparams)

Create and train the model as usual:

In [ ]:
# Trainer initialization
seq2seq_brain = seq2seqBrain(
    hparams["modules"],
    hparams["opt_class"],
    hparams,
    run_opts={"device": device},
)

# Training/validation loop
seq2seq_brain.fit(
    range(hparams["N_epochs"]),
    train_data,
    valid_data,
    train_loader_kwargs=hparams["dataloader_options"],
    valid_loader_kwargs=hparams["dataloader_options"],
)

100%|██████████| 2/2 [00:00<00:00,  4.93it/s]


Epoch 0 complete
Train loss: 3.69
Stage.VALID loss: 3.55
Stage.VALID PER: 85.96


100%|██████████| 2/2 [00:00<00:00,  4.96it/s]


Epoch 1 complete
Train loss: 3.14
Stage.VALID loss: 3.79
Stage.VALID PER: 89.47


100%|██████████| 2/2 [00:00<00:00,  5.22it/s]


...

Epoch 7 complete
Train loss: 0.81
Stage.VALID loss: 4.29
Stage.VALID PER: 84.21


100%|██████████| 2/2 [00:00<00:00,  5.20it/s]


Epoch 8 complete
Train loss: 0.65
Stage.VALID loss: 4.66
Stage.VALID PER: 87.72


100%|██████████| 2/2 [00:00<00:00,  5.15it/s]


Epoch 9 complete
Train loss: 0.53
Stage.VALID loss: 4.67
Stage.VALID PER: 84.21


Let's check out how good we did, shall we? :)

In [ ]:
print(seq2seq_brain.profiler.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::copy_        20.19%     230.314ms        20.19%     230.314ms     158.074us           0 b           0 b          1457  
                             aten::convolution_backward         9.22%     105.227ms        14.81%     168.933ms      42.233ms      15.49 Mb     -30.53 Mb             4  
                                          ProfilerStep*         8.77%     100.060ms        99.99%        1.141s     570.374ms     160.15 Kb     -28.74

Copying data takes time 🤔

Next, `evaluation()`:

In [ ]:
# Evaluation is run separately (now just evaluating on valid data)
seq2seq_brain.evaluate(valid_data)

100%|██████████| 2/2 [00:00<00:00,  3.94it/s]

Stage.TEST loss: 4.67
Stage.TEST PER: 84.21


4.668940544128418

Ok, let's check! 😎

In [ ]:
print(seq2seq_brain.profiler.key_averages().table(sort_by="cpu_time_total", row_limit=10))
len(seq2seq_brain.profiler.events())

Oh, no! 😞

Nothing—not enough samples in `valid_data` for the scheduler to warm-up.

Is all data lost? What happened to the results from profiling `fit()`?

In [ ]:
len(seq2seq_brain.profiler.speechbrain_event_traces[0])

46204

If a SpeechBrain model gets hooked to a profiler, the attribute `speechbrain_event_traces` is created. This is a list of previously recorded raw events. Sometimes one likes to revisit what happened previously but is not writing everything to the disk—here you have functionality.<br><i>Note: with `@profile_optimiser`, recorded traces are written to a `log` folder.</i>



In [ ]:
%ll log

total 19060
-rw-r--r-- 1 root 19515498 Jun 22 02:12 c0eb765b88a3_61.1655863932580.pt.trace.json


Back to topic.

If `evaluate()` would have had processed enough batches (so scheduler starts recording), then there would be a: `seq2seq_brain.profiler.speechbrain_event_traces[1]` with its recorded benchmark.

> Running `evaluate()` lots of times won't help – the scheduler always resets its counter to 0.
```python
    seq2seq_brain.evaluate(valid_data)
    seq2seq_brain.evaluate(valid_data)
    seq2seq_brain.evaluate(valid_data)
    seq2seq_brain.evaluate(valid_data)
    seq2seq_brain.evaluate(valid_data)
    seq2seq_brain.evaluate(valid_data)
```
No impact, not worth trying.

But, but – what to do instead?

```
NOT - using @profile to avoid the scheduler
      => this obtains another measurement as @profile_optimiser (worthless report)

N/A - reduce batch size, e.g.: 
      seq2seq_brain.evaluate(valid_data, test_loader_kwargs={"batch_size": 1})
      => problem in this specific case: 
         this will create two batches (two audio files in total); we need five or six batches

TRY - loop over dataset until scheduler records and has finished    

           ! NOT for _recognition performance_ reporting !
              might have the same numbers in this case
                      not a general guarantee
```

In [ ]:
seq2seq_brain.evaluate(sb.dataio.dataloader.make_dataloader(valid_data, looped_nominal_epoch=6))

100%|██████████| 6/6 [00:02<00:00,  2.18it/s]


Stage.TEST loss: 4.67
Stage.TEST PER: 84.21


4.668940544128418

In [ ]:
print([len(traces) for traces in seq2seq_brain.profiler.speechbrain_event_traces])

[46204, 21785]


In [ ]:
print(seq2seq_brain.profiler.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*        16.45%      97.250ms        99.98%     591.242ms     295.621ms     174.08 Kb    -184.38 Mb             2  
                                            aten::copy_        26.06%     154.120ms        26.06%     154.120ms     188.181us           0 b           0 b           819  
                                            aten::clone         0.12%     703.000us        25.89%     153.119ms       1.516ms      88.41 Mb           

What to say, copying/cloning takes time.<br/><i>Note: hints for optimisation potentials ;-)</i>

# 4. At-a-glance overviews for pretrained models

Summary reports on inference are demonstrated in [profiling recipe](https://github.com/anautsch/speechbrain/tree/benchmark/recipes/Profiling), which can be easily extended for representative testing and reporting (this scope is on researchers and developers). Two tables are created, one for the real-time factor, and another for the memory peak observed during inference.

It uses `@profile_report`: ten batches are run of which the last seven are recorded for benchmark. The extensive tables provide insights on averages; we report the upper control limit of real-time estimates.

To re-use the core functionality of the profiling recipe:

In [ ]:
!pip install transformers  # extra requirement for the speechbrain/asr-wav2vec2-commonvoice-fr pretrained model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 13.6 MB/s 
     |████████████████████████████████| 6.6 MB 38.6 MB/s 


In [ ]:
from tools.profiling.profile import profile_pretrained

profile_pretrained(pretrained_type="EncoderASR",
                   source = "speechbrain/asr-wav2vec2-commonvoice-fr",
                   save_dir = "pretrained_models/speechbrain/asr-wav2vec2-commonvoice-fr",
                   audio_mockup_secs = [1, 2],
                   batch_sizes = [1, 2],
                   triangle_only=False,  # True if the profile suite gets too heavy for your hardware
)

Downloading:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

speechbrain.lobes.models.huggingface_wav2vec - wav2vec 2.0 is frozen.


Downloading:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/238k [00:00<?, ?B/s]


Duration: 1, batch_size: 1
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   ProfilerStep*         4.16%     341.345ms       100.00%        8.215s        1.174s      32.00 Mb      -1.23 Gb             7  
                    aten::linear         0.20%      16.532ms        65.66%        5.395s       5.172ms     294.87 Mb           0 b          1043  
                    aten::matmul         0.51%      42.226ms        64.59%        5.306s       5.087ms     294.87 Mb      -4.02 Mb          1043  
                        aten::mm        63.77%        5.239s        63.78%        5.240s  

In one run, we observed on colab:


| Real-time factor |        1 |        2 |
|:--------:|:--------:|:--------:|
|      1s  | 1.56E+00 | 9.37E-01 |
|      2s  | 9.69E-01 | 8.40E-01 |

This E-[xponent] notation is formatted to easily spot if one is real-time, yet.
* 1.56E+00 = 1.56 --> not real-time
* 9.37E-01 = 0.937 -> real-time
* 9.69E-01 = 0.969 -> real-time
* 8.40E-01 has `E-01` and we see `-1 < 0` => real-time

The number after the `E`, the exponent, needs to be negative to claim real-time.

---

How about memory peaks?

| Peak memory |       1 |       2 |
|:-------:|:-------:|:-------:|
|     1s  | 0.09 Gb | 0.10 Gb |
|     2s  | 0.10 Gb | 0.18 Gb 

***Note:*** this is solely about observations during and because of inference: your code might push more to the VRAM as is included here.

# There's more!
Please check out our [unittests](https://github.com/anautsch/speechbrain/blob/benchmark/tests/unittests/test_profiling.py) which have about a 1,000 lines of commented tutorial-style code.

# **About SpeechBrain**
- Website: https://speechbrain.github.io/
- Code: https://github.com/speechbrain/speechbrain/
- HuggingFace: https://huggingface.co/speechbrain/


# **Citing SpeechBrain**
Please, cite SpeechBrain if you use it for your research or business.

```bibtex
@misc{speechbrain,
  title={{SpeechBrain}: A General-Purpose Speech Toolkit},
  author={Mirco Ravanelli and Titouan Parcollet and Peter Plantinga and Aku Rouhe and Samuele Cornell and Loren Lugosch and Cem Subakan and Nauman Dawalatabad and Abdelwahab Heba and Jianyuan Zhong and Ju-Chieh Chou and Sung-Lin Yeh and Szu-Wei Fu and Chien-Feng Liao and Elena Rastorgueva and François Grondin and William Aris and Hwidong Na and Yan Gao and Renato De Mori and Yoshua Bengio},
  year={2021},
  eprint={2106.04624},
  archivePrefix={arXiv},
  primaryClass={eess.AS},
  note={arXiv:2106.04624}
}
```